<a href="https://colab.research.google.com/github/GustavinhoDantas/Verificacao-duplicidades/blob/main/Duplicidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DUPLICIDADES



In [42]:
pip install networkx

In [62]:
import pandas as pd
import numpy as np

In [63]:
# Caminhos dos arquivos
arquivo_subir = '/content/Subir_IBI_BV.xlsx'
arquivo_api = '/content/Lote_258_2025_06_11_10_32_52.xlsx'
arquivo_portal = '/content/query_result_2025-06-11T11_48_14.593598-03_00.xlsx'

arq_subir = pd.read_excel(arquivo_subir)
arq_api = pd.read_excel(arquivo_api)
arq_portal = pd.read_excel(arquivo_portal)

print('lidas')

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


lidas


In [ ]:
# Acesso seguro às colunas com strip
arq_subir['Apólice'] = arq_subir['Apólice'].str.strip()
arq_api['Número da Apólice'] = arq_api['Número da Apólice'].str.strip()
arq_portal['Apólice'] = arq_portal['Apólice'].str.strip()
arq_portal['Apólice renovada'] = arq_portal['Apólice renovada'].str.strip()


print('Polidas')

Polidas


In [ ]:
# Vendo se a apolice em questão já não está no portal, batendo apolice x apolice e apolice  x endosso ]
mask_importadas_apolice = ( arq_portal['Apólice'].isin(arq_subir['Apólice']) &
                            arq_portal['Endosso'].isna()
)

mask_importadas_endossos = arq_portal['Endosso'].isin(arq_subir['Apólice'])

mask_importadas = mask_importadas_apolice | mask_importadas_endossos

colunas_resultado = ['ID AVITA', 'Apólice', 'Endosso']
portal_filtrado = arq_portal.loc[mask_importadas, colunas_resultado]

# Remove duplicatas, se necessário
portal_filtrado = portal_filtrado.drop_duplicates()

# Salva no Excel
portal_filtrado.to_excel('z_01_subir_ja_no_portal.xlsx', index=False)
print('Subir já no portal salvo com sucesso!')

Subir já no portal salvo com sucesso!


In [ ]:
# Conjunto de apólices da SUBIR que já estão no portal
apolices_ja_no_portal = set(arq_portal.loc[mask_importadas_apolice, 'Apólice']) | set(arq_portal.loc[mask_importadas_endossos, 'Endosso'])

# Remove da SUBIR essas apólices
arq_subir_filtrado = arq_subir[~arq_subir['Apólice'].isin(apolices_ja_no_portal)]

# Salvar resultado
arq_subir_filtrado.to_excel('z_02_subir_filtrado.xlsx', index=False)
print('Base de subir filtrada com sucesso!')

Base de subir filtrada com sucesso!


In [ ]:
# Criar uma máscara booleana para linhas da arq_api onde a apólice está em arq_subir['Apólice']
mask_api = arq_api['Número da Apólice'].isin(arq_subir_filtrado['Apólice'])
correspondentes_api = arq_api.loc[mask_api]

print('Vendo correspondencias')

#df = pd.DataFrame(correspondentes)
#df.to_excel('teste.xlsx', index=False)
#print('Dados extraídos e salvos no Excel!')

Vendo correspondencias


In [ ]:
#Pegando a correspondencia, mas apenas que são renovações
renovacao = correspondentes_api[correspondentes_api['Renovação?'] == 'sim']
print('Vista apenas as renovações')

#tira a repetição da mesma apolice
renovacao = renovacao.drop_duplicates(subset='Número da Apólice', keep='first')
print('duplicidades')

cancelados = renovacao[renovacao['Status'] == 'cancelado']
renovacao = renovacao[renovacao['Status'] != 'cancelado']

df = pd.DataFrame(renovacao)
cancelados.to_excel('z_03_cancelados_seguradora.xlsx', index=False)
df.to_excel('z_04_renovacao.xlsx', index=False)
print('Dados extraídos e salvos no Excel!')

Vista apenas as renovações
duplicidades
Dados extraídos e salvos no Excel!


Agora fazendo o bate com o portal

In [64]:
#arrumar para pegar os endossos dessas apolice então buscar na campo de apolice renovada do portal, ou seja, bater endosso do portal x apolice

# Garantir que os valores estejam padronizados como strings e sem espaços
renovacao['Número da Apólice Renovada'] = renovacao['Número da Apólice Renovada'].astype(str).str.strip()
arq_portal['Apólice'] = arq_portal['Apólice'].astype(str).str.strip()
arq_portal['Apólice renovada'] = arq_portal['Apólice renovada'].astype(str).str.strip()

# Máscara correta: procurar na tabela portal os valores que estão na renovacao
mask_portal = (
    arq_portal['Apólice'].isin(renovacao['Número da Apólice Renovada']) |
    arq_portal['Apólice renovada'].isin(renovacao['Número da Apólice Renovada']) |
    arq_portal['Endosso'].isin(renovacao['Número da Apólice Renovada'])
)
# Filtrar as linhas da tabela do portal
correspondentes_portal = arq_portal[mask_portal]

# 1. Pega todos os endossos da tabela correspondentes_portal e ve se tem na coluna apolice renovada no portal
endossos_encontrados = correspondentes_portal['Endosso'].dropna().unique()
mask_endosso_renovado = arq_portal['Apólice renovada'].isin(endossos_encontrados)
linhas_expandidas = arq_portal[mask_endosso_renovado].copy()
trilha= pd.concat([correspondentes_portal, linhas_expandidas]).drop_duplicates().reset_index(drop=True)

#tabela onde vai os ID_Trilha e já adiciona o mesmo ID quando a apolice aparece mais de uma vez na coluna
trilha['ID_Trilha'] = trilha['Apólice'].astype('category').cat.codes + 1


trilha['Endosso'] = trilha['Endosso'].replace(['nan', 'NaN', 'None', ''], np.nan)
# Criar dicionário apenas com Endossos válidos (não nulos)
map_endosso_para_id = trilha.dropna(subset=['Endosso']).set_index('Endosso')['ID_Trilha'].to_dict()

# Aplicar substituição do ID_Trilha se o valor da apólice renovada estiver no dicionário de endossos
trilha['ID_Trilha'] = trilha.apply(
    lambda row: map_endosso_para_id[row['Apólice renovada']]
    if row['Apólice renovada'] in map_endosso_para_id
    else row['ID_Trilha'],
    axis=1
)


# Salvar no Excel
trilha.to_excel('z_05_trilha.xlsx', index=False)
print('Linhas do portal que batem com renovações salvas no Excel!')

Linhas do portal que batem com renovações salvas no Excel!


In [ ]:
# 1. Filtrar no portal as linhas que batem com a renovação
portal_correspondente = arq_portal.loc[mask_portal].copy()

# 2. Filtrar só os cancelados (considerando que a coluna Status está na tabela portal)
cancelados_portal = portal_correspondente[portal_correspondente['Status'].str.lower() == 'cancelado']

# 3. Agora, pegar as apólices canceladas do portal para marcar na renovacao
apolices_canceladas_no_portal = cancelados_portal['Apólice'].unique()

# 4. Criar coluna em renovacao indicando se está cancelado no portal
# Vamos considerar que a chave para ligação é 'Número da Apólice Renovada' na renovacao
renovacao['Cancelado no portal'] = renovacao['Número da Apólice Renovada'].apply(
    lambda x: 'Sim' if x in apolices_canceladas_no_portal else 'Não'
)

# 5. Salvar ou conferir
renovacao.to_excel('renovacao_com_cancelados_portal.xlsx', index=False)
print('Arquivo salvo com informação de cancelados no portal na tabela renovacao.')

Arquivo salvo com informação de cancelados no portal na tabela renovacao.


In [39]:
def verifica_duplicidade_trilha(row):
    apolice_mae = row['Número da Apólice Renovada']
    # Filtra a trilha: todas as linhas do portal com 'Apólice' igual
    trilha = arq_portal[arq_portal['Apólice'] == apolice_mae].copy()

    if trilha.empty:
        return 'Trilha não encontrada'

    # Apólice mãe: linha da trilha com Endosso vazio ou NaN
    apolice_mae_row = trilha[trilha['Endosso'].isna() | (trilha['Endosso'] == '')]

    if apolice_mae_row.empty:
        return 'Apólice mãe não identificada'

    # Data da apólice mãe (final de vigência)
    data_mae = pd.to_datetime(apolice_mae_row.iloc[0]['Final Vigência'])

    # Datas únicas da trilha (final de vigência)
    datas_trilha = pd.to_datetime(trilha['Final Vigência'].dropna().unique())

    # Se alguma data na trilha for diferente da data da apólice mãe => possível duplicidade
    if any(data != data_mae for data in datas_trilha):
        return 'Possível duplicidade'
    else:
        return 'OK para importação'

# Aplica a função para cada linha da tabela renovacao
renovacao['Status Trilhas'] = renovacao.apply(verifica_duplicidade_trilha, axis=1)

# Salva resultado para conferência
renovacao.to_excel('renovacao_com_status_trilhas.xlsx', index=False)
print('Arquivo renovacao_com_status_trilhas.xlsx salvo com a coluna Status Trilhas.')

Arquivo renovacao_com_status_trilhas.xlsx salvo com a coluna Status Trilhas.



- Bater a SUBIR já no portal (apolice x apolice e apolice x endosso) - fazer com que retorne uma tabela com campus ID portal|Apolice|Endosso
- Maneira de bater a planilha renovacao com a do portal para ver duplicidades, para isso é necessario ver a trilha (trilha é aquela que tem vinculo com a mesma apolice, ou seja, tudo que tiver na base do portal com a coluna "Apolice" igual é a trilha) para ver isso é necessario bater as datas, se a data de final de vigencia da trilha for igual a da apolice tudo bem, caso seja diferente houve uma renovação, por isso é uma duplicidade

